# **Can you recommend  jokes that users may like?**

# **1.1 Initialize**

In [23]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357293 sha256=d38acea4bd30defd7f7911c6fc7b0a6193d90fd7b0d2b89b043ddac5b3a50d7a
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split, cross_validate,GridSearchCV


# **1.2 Read the Data**

Movie metadata

In [2]:
df = pd.read_csv('data.csv')

User ratings for each film.

In [3]:
dfr1 = pd.read_csv('UserRatings1.csv')

In [4]:
dfr2 = pd.read_csv('UserRatings2.csv')

<ipython-input-4-589955785acd>:1: DtypeWarning: Columns (18477) have mixed types. Specify dtype option on import or set low_memory=False.
  dfr2 = pd.read_csv('UserRatings2.csv')


See first few rows in the data.

# **1.3 Prepare the Data**

In [5]:
df.head()

,JokeId,JokeText
0,0,"A man visits the doctor. The doctor says ""I ha..."
1,1,This couple had an excellent relationship goin...
2,2,Q. What's 200 feet long and has 4 teeth? \n\nA...
3,3,Q. What's the difference between a man and a t...
4,4,Q.\tWhat's O. J. Simpson's Internet address? \...


In [6]:
dfr1.head()

,JokeId,User1,User2,User3,User4,User5,User6,User7,User8,User9,...,User36701,User36702,User36703,User36704,User36705,User36706,User36707,User36708,User36709,User36710
0,0,5.10,-8.79,-3.50,7.14,-8.79,9.22,-4.03,3.11,-3.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.91,NaN
1,1,4.90,-0.87,-2.91,-3.88,-0.58,9.37,-1.55,0.92,-3.35,...,NaN,NaN,NaN,-5.63,NaN,-6.07,NaN,-1.60,-4.56,NaN
2,2,1.75,1.99,-2.18,-3.06,-0.58,-3.93,-3.64,7.52,-6.46,...,NaN,NaN,NaN,NaN,NaN,4.08,NaN,NaN,8.98,NaN
3,3,-4.17,-4.61,-0.10,0.05,8.98,9.27,-6.99,0.49,-3.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5.15,5.39,7.52,6.26,7.67,3.45,5.44,-0.58,1.26,...,2.28,-0.49,5.1,-0.29,-3.54,-1.36,7.48,-5.78,0.73,2.62


In [7]:
dfr2.head()

,JokeId,User36711,User36712,User36713,User36714,User36715,User36716,User36717,User36718,User36719,...,User73412,User73413,User73414,User73415,User73416,User73417,User73418,User73419,User73420,User73421
0,0,NaN,NaN,NaN,3.93,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5.68,1.07,8.11,-2.33,-5.83,8.2,-5.83,1.94,0.10,...,3.64,4.32,6.99,-9.66,-8.4,-0.63,9.51,-7.67,-1.6,8.3


In [11]:
# Check for missing values in user ratings datasets
dfr1.fillna(dfr1.mean(numeric_only=True), inplace=True)
dfr2.fillna(dfr2.mean(numeric_only=True), inplace=True)
# Check for missing values in jester data
df.fillna(df.mean(numeric_only=True), inplace=True)

In [12]:
# Display the shape of each dataset
print("User Ratings 1 (dfr1) shape:", dfr1.shape)
print("User Ratings 2 (dfr2) shape:", dfr2.shape)
print("Jester Data (df) shape:", df.shape)

User Ratings 1 (dfr1) shape: (37, 36711)
User Ratings 2 (dfr2) shape: (90, 36712)
Jester Data (df) shape: (100, 2)


As seen above, the user ratings data has 37 users but 36,711 ratings. While there are a large number of ratings, the user count is quite low.

# **Build Recommendations**

If there are duplicates, handle them.

In [13]:
# Check for duplicates in dfr1
duplicates_dfr1 = dfr1.duplicated()
print("Duplicates in User Ratings 1 (dfr1):", duplicates_dfr1.sum())

# Check for duplicates in dfr2
duplicates_dfr2 = dfr2.duplicated()
print("Duplicates in User Ratings 2 (dfr2):", duplicates_dfr2.sum())

# Check for duplicates in df
duplicates_df = df.duplicated()
print("Duplicates in Jester Data (df):", duplicates_df.sum())


Duplicates in User Ratings 1 (dfr1): 0
Duplicates in User Ratings 2 (dfr2): 0
Duplicates in Jester Data (df): 0


There are no duplicates in the data sets, so work directly on data sets.

# **Build Model**

**Content Based Filtering**

In [21]:
# TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['JokeText'])

# Calculate cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

titles = df['JokeText']
indices = pd.Series(df.index, index=df['JokeText'])

# Define the recommendation function
def get_recommendations(joke_text, cosine_sim=cosine_sim):
    idx = indices[joke_text]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    joke_indices = [i[0] for i in sim_scores]
    return titles.iloc[joke_indices]

# Give a example of it
example_joke_text = df['JokeText'].iloc[0]  # Örnek bir şaka metni seçiyoruz
print("Recommended jokes based on the example joke:")
print(get_recommendations(example_joke_text))



Recommended jokes based on the example joke:
86    A man, recently completing a routine physical ...
67    A man piloting a hot air balloon discovers he ...
87    A Czechoslovakian man felt his eyesight was gr...
75    There once was a man and a woman that both  go...
31    A man arrives at the gates of heaven. St. Pete...
38    What is the difference between men and women:\...
55    A man and Cindy Crawford get stranded on a des...
80    An Asian man goes into a New York CityBank to ...
32    What do you call an American in the finals of ...
3     Q. What's the difference between a man and a t...
Name: JokeText, dtype: object


This method makes recommendations based solely on the current content information without considering past ratings. It does this by examining the similarities between joke texts.

 **Collaborative Filtering**

Collaborative filtering, unlike content-based filtering, makes recommendations based on information collected from past user interactions.

Prepare the data

In [31]:
# Convert data to long format
dfr1_melted = dfr1.melt(id_vars=['JokeId'], var_name='userId', value_name='rating')
dfr2_melted = dfr2.melt(id_vars=['JokeId'], var_name='userId', value_name='rating')

# userId is converted
dfr1_melted['userId'] = dfr1_melted['userId'].str.extract('(\d+)').astype(int)
dfr2_melted['userId'] = dfr2_melted['userId'].str.extract('(\d+)').astype(int)

print("Melted and cleaned data for dfr1:")
print(dfr1_melted.head())
print("Melted and cleaned data for dfr2:")
print(dfr2_melted.head())


Melted and cleaned data for dfr1:
   JokeId  userId  rating
0       0       1    5.10
1       1       1    4.90
2       2       1    1.75
3       3       1   -4.17
4       4       1    5.15
Melted and cleaned data for dfr2:
   JokeId  userId    rating
0       0   36711  4.023333
1       1   36711  4.023333
2       2   36711  4.023333
3       3   36711  4.023333
4       4   36711      5.68


In [33]:
# Replace hyphens with NaN and then convert to numeric
dfr1_melted['rating'] = pd.to_numeric(dfr1_melted['rating'].replace('-', None), errors='coerce')
dfr2_melted['rating'] = pd.to_numeric(dfr2_melted['rating'].replace('-', None), errors='coerce')

# Begin the reader
reader = Reader(rating_scale=(-10, 10))

# Add data to surprise
dataset1 = Dataset.load_from_df(dfr1_melted[['userId', 'JokeId', 'rating']], reader)
dataset2 = Dataset.load_from_df(dfr2_melted[['userId', 'JokeId', 'rating']], reader)

In [34]:
# split the data into test and train sets
trainset1, testset1 = train_test_split(dataset1, test_size=0.25)
trainset2, testset2 = train_test_split(dataset2, test_size=0.25)


In [35]:
# SVD is defined
svd1 = SVD()
svd2 = SVD()

# Train the model
svd1.fit(trainset1)
svd2.fit(trainset2)


In [40]:
# Test the model and RMSE calculation
predictions1 = svd1.test(testset1)
predictions2 = svd2.test(testset2)

rmse1 = accuracy.rmse(predictions1)
rmse2 = accuracy.rmse(predictions2)

print(f"RMSE for User Ratings 1 (dfr1): {rmse1}")
print(f"RMSE for User Ratings 2 (dfr2): {rmse2}")


RMSE: 4.1770
RMSE: nan
RMSE for User Ratings 1 (dfr1): 4.176953213590095
RMSE for User Ratings 2 (dfr2): nan


In [45]:
# Hyperparameter optimization
param_grid = {'n_epochs': [5, 10, 15], 'lr_all': [0.002, 0.005], 'reg_all': [0.4, 0.6]}
gs1 = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs2 = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs1.fit(dataset1)
gs2.fit(dataset2)

# Best RMSE
print(f"Best RMSE for User Ratings 1 (dfr1): {gs1.best_score['rmse']}")
print(f"Best RMSE for User Ratings 2 (dfr2): {gs2.best_score['rmse']}")

# Best RMSE score and its parameters
print(f"Best parameters for User Ratings 1 (dfr1): {gs1.best_params['rmse']}")
print(f"Best parameters for User Ratings 2 (dfr2): {gs2.best_params['rmse']}")


Best RMSE for User Ratings 1 (dfr1): 4.470860747169224
Best RMSE for User Ratings 2 (dfr2): nan
Best parameters for User Ratings 1 (dfr1): {'n_epochs': 15, 'lr_all': 0.002, 'reg_all': 0.4}
Best parameters for User Ratings 2 (dfr2): {'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}


In [46]:
# Cross validation
cross_validate(svd1, dataset1, measures=['RMSE', 'MAE'], cv=5, verbose=True)
cross_validate(svd2, dataset2, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    4.1890  4.1869  4.1804  4.1852  4.1851  4.1853  0.0029  
MAE (testset)     3.2743  3.2731  3.2706  3.2756  3.2750  3.2737  0.0018  
Fit time          19.65   20.37   20.56   21.06   21.24   20.58   0.56    
Test time         1.69    1.68    5.08    5.16    1.76    3.08    1.67    
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    nan     10.2944 10.3000 10.3009 10.3016 nan     nan     
MAE (testset)     nan     9.5496  9.5537  9.5546  9.5565  nan     nan     
Fit time          52.25   54.11   54.34   53.92   54.73   53.87   0.86    
Test time         11.31   9.00    12.54   8.68    8.54    10.01   1.61    


{'test_rmse': array([        nan, 10.29438803, 10.30001818, 10.30093478, 10.30164202]),
 'test_mae': array([       nan, 9.54962065, 9.55368124, 9.55461636, 9.55654922]),
 'fit_time': (52.24648404121399,
  54.11423397064209,
  54.33638000488281,
  53.92184638977051,
  54.73293662071228),
 'test_time': (11.314186811447144,
  8.998346328735352,
  12.536261796951294,
  8.677438259124756,
  8.543843984603882)}

In [47]:
# Give suggestions for user
def recommend_jokes(model, user_id, num_recommendations=5):
    # Get the joke id's
    joke_ids = dfr1_melted['JokeId'].unique()

    # Find the jokes that users did not rate
    user_rated_jokes = dfr1_melted[dfr1_melted['userId'] == user_id]['JokeId'].unique()
    not_rated_jokes = set(joke_ids) - set(user_rated_jokes)

    # Predictions
    predictions = [model.predict(user_id, joke_id) for joke_id in not_rated_jokes]
    predictions.sort(key=lambda x: x.est, reverse=True)

    # Top recommendations
    top_jokes = predictions[:num_recommendations]
    return [(pred.iid, pred.est) for pred in top_jokes]

# Example usage
user_id = 1
recommendations1 = recommend_jokes(svd1, user_id)
recommendations2 = recommend_jokes(svd2, user_id)

print(f"Top {len(recommendations1)} recommendations for user {user_id} in User Ratings 1 (dfr1):")
for joke_id, rating in recommendations1:
    print(f"Joke ID: {joke_id}, Predicted Rating: {rating}")

print(f"Top {len(recommendations2)} recommendations for user {user_id} in User Ratings 2 (dfr2):")
for joke_id, rating in recommendations2:
    print(f"Joke ID: {joke_id}, Predicted Rating: {rating}")


Top 0 recommendations for user 1 in User Ratings 1 (dfr1):
Top 0 recommendations for user 1 in User Ratings 2 (dfr2):


In [48]:
# Predict rating function
def predict_rating(model, user_id, joke_id):
    pred = model.predict(user_id, joke_id)
    return pred.est

# Predict ratings for user
user_id = 1
joke_id = 302

predicted_rating1 = predict_rating(svd1, user_id, joke_id)
predicted_rating2 = predict_rating(svd2, user_id, joke_id)

print(f"Predicted rating for user {user_id} on joke {joke_id} in User Ratings 1 (dfr1): {predicted_rating1}")
print(f"Predicted rating for user {user_id} on joke {joke_id} in User Ratings 2 (dfr2): {predicted_rating2}")


Predicted rating for user 1 on joke 302 in User Ratings 1 (dfr1): 0.5592046558911319
Predicted rating for user 1 on joke 302 in User Ratings 2 (dfr2): 10
